# Import dependencies

In [18]:
import os
import sys

sys.path.insert(0, os.path.dirname(os.getcwd())) 

In [19]:
import time
import gc
import json

import numpy as np
import pandas as pd

from transformers import BertTokenizer, BertModel
from transformers import logging

from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score

from InputDataset import InputDataset

import torch
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from torch.nn.utils.rnn import pad_sequence

from torch import cuda

from models.BERT_Dropout_BiLSTM_Linear import BERT_Dropout_BiLSTM_Linear

In [20]:
device = 'cuda' if cuda.is_available() else 'cpu'
logging.set_verbosity_error() 

In [21]:
print(torch.cuda.get_device_name(0))
print(f"Memory: {torch.cuda.get_device_properties(0).total_memory // 1024 ** 3} GB")

NVIDIA GeForce RTX 2060 SUPER
Memory: 8 GB


In [22]:
def clear_memory():
    torch.cuda.empty_cache()

    with torch.no_grad():
        torch.cuda.empty_cache()

    gc.collect()

# Load Data

In [23]:
DATASET = 'ATE_SemEval16_Restaurants_train.json'

In [24]:
df = pd.json_normalize(json.load(open(DATASET)))

In [25]:
df.head()

,text,tokens,iob_aspect_tags
0,Judging from previous posts this used to be a ...,"[Judging, from, previous, posts, this, used, t...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ..."
1,"We, there were four of us, arrived at noon - t...","[We, ,, there, were, four, of, us, ,, arrived,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,"They never brought us complimentary noodles, i...","[They, never, brought, us, complimentary, nood...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,The food was lousy - too sweet or too salty an...,"[The, food, was, lousy, -, too, sweet, or, too...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]"
4,The food was lousy - too sweet or too salty an...,"[The, food, was, lousy, -, too, sweet, or, too...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]"


# Train & Validate

In [26]:
TRAIN_BATCH_SIZE = 4
VALID_BATCH_SIZE = 4

EPOCHS = 2

LEARNING_RATE = 1e-5

TRAIN_SPLIT = 0.8

NO_RUNS = 10

In [27]:
MODEL_OUTPUT = '../../../results/ATE/SemEval16 - Task 5 - Restaurants/models/bert_pre_trained_dropout_bilstm_linear.pth'
STATS_OUTPUT = '../../../results/ATE/SemEval16 - Task 5 - Restaurants/stats/bert_pre_trained_dropout_bilstm_linear.csv'

In [28]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [29]:
best_accuracy = 0.0

In [30]:
def create_mini_batch(samples):
    ids_tensors = [s[1] for s in samples]
    ids_tensors = pad_sequence(ids_tensors, batch_first=True).to(device)

    tags_tensors = [s[2] for s in samples]
    tags_tensors = pad_sequence(tags_tensors, batch_first=True).to(device)
    
    masks_tensors = torch.zeros(ids_tensors.shape, dtype=torch.long).to(device)
    masks_tensors = masks_tensors.masked_fill(ids_tensors != 0, 1).to(device)
    
    return ids_tensors, tags_tensors, masks_tensors

In [31]:
def train(epoch, model, loss_fn, optimizer, dataloader):
    model.train()

    dataloader_len = len(dataloader)

    for _,data in enumerate(dataloader, 0):
        optimizer.zero_grad()

        ids_tensors, tags_tensors, masks_tensors = data

        outputs = model(ids_tensors, masks_tensors)

        loss = loss_fn(outputs.view(-1, 3), tags_tensors.view(-1))
        
        if _ % (dataloader_len // 10) == 0:
            print(f"Epoch: {epoch}/{EPOCHS}, Batch: {_}/{dataloader_len}, Loss: {loss.item()}")
        
        loss.backward()
        
        optimizer.step()

In [32]:
def validation(model, dataloader):
    model.eval()
    
    fin_targets=[]
    fin_outputs=[]

    with torch.no_grad():
        for _, data in enumerate(dataloader, 0):
            ids_tensors, tags_tensors, masks_tensors = data
            ids_tensors = ids_tensors.to(device)
            tags_tensors = tags_tensors.to(device)
            masks_tensors = masks_tensors.to(device)

            outputs = model(ids_tensors, masks_tensors)
            
            _, predictions = torch.max(outputs, dim=2)

            fin_outputs += list([int(p) for pred in predictions for p in pred])
            fin_targets += list([int(tag) for tags_tensor in tags_tensors for tag in tags_tensor])

    return fin_outputs, fin_targets

In [33]:
results = pd.DataFrame(columns=['accuracy','precision_score_micro','precision_score_macro','recall_score_micro','recall_score_macro','f1_score_micro','f1_score_macro', 'execution_time'])

In [34]:
for i in range(NO_RUNS):
    # clear cache cuda
    torch.cuda.empty_cache()
    with torch.no_grad():
        torch.cuda.empty_cache()
    gc.collect()

    start_time = time.time()

    print(f"Run {i + 1}/{NO_RUNS}")

    train_dataset = df.sample(frac=TRAIN_SPLIT)
    test_dataset = df.drop(train_dataset.index).reset_index(drop=True)
    train_dataset = train_dataset.reset_index(drop=True)

    training_set = InputDataset(train_dataset, tokenizer)
    testing_set = InputDataset(test_dataset, tokenizer)

    train_dataloader = DataLoader(
        training_set,
        sampler = RandomSampler(train_dataset),
        batch_size = TRAIN_BATCH_SIZE,
        drop_last = True,
        collate_fn=create_mini_batch
    )

    validation_dataloader = DataLoader(
        testing_set,
        sampler = SequentialSampler(testing_set),
        batch_size = VALID_BATCH_SIZE,
        drop_last = True,
        collate_fn=create_mini_batch
    )

    model = BERT_Dropout_BiLSTM_Linear(BertModel.from_pretrained('bert-base-uncased'), dropout=0.3, bilstm_in_features=256, no_out_labels=3, device=device).to(device)

    optimizer = torch.optim.Adam(params = model.parameters(), lr=LEARNING_RATE)
    loss_fn = torch.nn.CrossEntropyLoss()

    for epoch in range(EPOCHS):
        train(epoch, model, loss_fn, optimizer, train_dataloader)

    outputs, targets = validation(model, validation_dataloader)
    
    accuracy = accuracy_score(targets, outputs)
    precision_score_micro = precision_score(targets, outputs, average='micro')
    precision_score_macro = precision_score(targets, outputs, average='macro')
    recall_score_micro = recall_score(targets, outputs, average='micro')
    recall_score_macro = recall_score(targets, outputs, average='macro')
    f1_score_micro = f1_score(targets, outputs, average='micro')
    f1_score_macro = f1_score(targets, outputs, average='macro')

    execution_time = time.time() - start_time

    results.loc[i] = [accuracy,precision_score_micro,precision_score_macro,recall_score_micro,recall_score_macro,f1_score_micro,f1_score_macro, execution_time]

    if accuracy > best_accuracy:
        best_accuracy = accuracy
        torch.save(model.bert, MODEL_OUTPUT)

    del train_dataset
    del test_dataset
    del training_set
    del testing_set
    del model
    del loss_fn
    del optimizer
    del outputs
    del targets

Run 1/10
Epoch: 0/2, Batch: 0/501, Loss: 1.1339452266693115
Epoch: 0/2, Batch: 50/501, Loss: 0.5134332180023193
Epoch: 0/2, Batch: 100/501, Loss: 0.39175689220428467
Epoch: 0/2, Batch: 150/501, Loss: 0.15073518455028534
Epoch: 0/2, Batch: 200/501, Loss: 0.18087317049503326
Epoch: 0/2, Batch: 250/501, Loss: 0.19568811357021332
Epoch: 0/2, Batch: 300/501, Loss: 0.22810862958431244
Epoch: 0/2, Batch: 350/501, Loss: 0.20390141010284424
Epoch: 0/2, Batch: 400/501, Loss: 0.2804506719112396
Epoch: 0/2, Batch: 450/501, Loss: 0.13913506269454956
Epoch: 0/2, Batch: 500/501, Loss: 0.05047709494829178
Epoch: 1/2, Batch: 0/501, Loss: 0.1428849697113037
Epoch: 1/2, Batch: 50/501, Loss: 0.03153345361351967
Epoch: 1/2, Batch: 100/501, Loss: 0.08569502085447311
Epoch: 1/2, Batch: 150/501, Loss: 0.19120566546916962
Epoch: 1/2, Batch: 200/501, Loss: 0.04239630699157715
Epoch: 1/2, Batch: 250/501, Loss: 0.028872821480035782
Epoch: 1/2, Batch: 300/501, Loss: 0.1451580673456192
Epoch: 1/2, Batch: 350/501, L

In [35]:
results

,accuracy,precision_score_micro,precision_score_macro,recall_score_micro,recall_score_macro,f1_score_micro,f1_score_macro,execution_time
0,0.968973,0.968973,0.824030,0.968973,0.826076,0.968973,0.823478,114.479259
1,0.963371,0.963371,0.838970,0.963371,0.801146,0.963371,0.798742,110.475375
2,0.970810,0.970810,0.848723,0.970810,0.826299,0.970810,0.836264,107.795127
3,0.966620,0.966620,0.803690,0.966620,0.843743,0.966620,0.821871,113.568082
4,0.967966,0.967966,0.827648,0.967966,0.832434,0.967966,0.827483,113.804849
5,0.970737,0.970737,0.849157,0.970737,0.842819,0.970737,0.844711,108.592092
6,0.954054,0.954054,0.756217,0.954054,0.668436,0.954054,0.639836,111.354443
7,0.971528,0.971528,0.850040,0.971528,0.850557,0.971528,0.850297,111.676461
8,0.964305,0.964305,0.800726,0.964305,0.865857,0.964305,0.816864,110.984551
9,0.969122,0.969122,0.821398,0.969122,0.808392,0.969122,0.806400,109.699027


In [36]:
results.to_csv(STATS_OUTPUT)